Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [52]:
batch_size = 128
beta_regul = 1e-3

def forward_pass(dataset, weights, biases):
    relu_hidden_layer = tf.nn.relu(tf.add(tf.matmul(dataset, weights['h1']), biases['b1']))
    out_layer = tf.matmul(relu_hidden_layer, weights['out']) + biases['out']
    return out_layer

mlp_regularized_graph = tf.Graph()
with mlp_regularized_graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Store layers weight & bias
    n_input = image_size * image_size
    n_hidden_1 = 1024
    n_classes = num_labels
    
    weights = {
        'h1': tf.Variable(tf.truncated_normal([n_input, n_hidden_1])),
        'out': tf.Variable(tf.truncated_normal([n_hidden_1, n_classes]))
    }
    
    biases = {
        'b1': tf.Variable(tf.zeros([n_hidden_1])),
        'out': tf.Variable(tf.zeros([n_classes]))
    }
    
    out_layer = forward_pass(tf_train_dataset, weights, biases)
    
    # Yes, those probably should be used as placeholder values, but ... FIXME
    test_out_layer = forward_pass(tf_test_dataset, weights, biases)
    valid_out_layer = forward_pass(tf_valid_dataset, weights, biases)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(out_layer, tf_train_labels)) # Softmax loss
    
    # L2 Regularization
    regularizers = (tf.nn.l2_loss(weights['h1']) + tf.nn.l2_loss(weights['out']))
    
    # Add the regularization term to the loss. Let's fix regularization parameter for the sake of simplicity
    loss += 5e-4 * regularizers
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
# train and predict
def train_and_test(graph, train_dataset, num_steps):
    '''Train using mini batch gradient descent'''
    with tf.Session(graph = graph) as session:
        tf.initialize_all_variables().run()
        print("initialization finished")

        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}

            _, l, predictions = session.run([optimizer, loss, out_layer], feed_dict=feed_dict)
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(valid_out_layer.eval(), valid_labels))
              print("Test accuracy: %.1f%%" % accuracy(test_out_layer.eval(), test_labels))
                
train_and_test(mlp_regularized_graph, train_dataset, 100)

initialization finished
Minibatch loss at step 0: 489.824158
Minibatch accuracy: 7.8%
Validation accuracy: 28.9%
Test accuracy: 31.2%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [53]:
train_and_test(mlp_regularized_graph, train_dataset, 100)

initialization finished
Minibatch loss at step 0: 532.448914
Minibatch accuracy: 11.7%
Validation accuracy: 35.9%
Test accuracy: 39.3%


Validation - test accuracy ~ 3 %

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [50]:
def dropout_forward_pass(dataset, weights, biases, keep_prob):
    relu_hidden_layer = tf.nn.relu(tf.add(tf.matmul(dataset, weights['h1']), biases['b1']))
    relu_hidden_layer_drop = tf.nn.dropout(relu_hidden_layer, keep_prob)
    out_layer = tf.matmul(relu_hidden_layer, weights['out']) + biases['out']
    return out_layer

mlp_dropout_graph = tf.Graph()
with mlp_dropout_graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    keep_prob = tf.placeholder("float")
    
    # Store layers weight & bias
    n_input = image_size * image_size
    n_hidden_1 = 1024
    n_classes = num_labels
    
    weights = {
        'h1': tf.Variable(tf.truncated_normal([n_input, n_hidden_1])),
        'out': tf.Variable(tf.truncated_normal([n_hidden_1, n_classes]))
    }
    
    biases = {
        'b1': tf.Variable(tf.zeros([n_hidden_1])),
        'out': tf.Variable(tf.zeros([n_classes]))
    }
    
    out_layer = dropout_forward_pass(tf_train_dataset, weights, biases, 0.5)
    
    # Yes, those probably should be used as placeholder values, but ... FIXME
    test_out_layer = dropout_forward_pass(tf_test_dataset, weights, biases, 0.5)
    valid_out_layer = dropout_forward_pass(tf_valid_dataset, weights, biases, 0.5)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(out_layer, tf_train_labels)) # Softmax loss
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
train_and_test(mlp_dropout_graph, train_dataset, 100)

initialization finished
Minibatch loss at step 0: 425.155701
Minibatch accuracy: 14.8%
Validation accuracy: 31.1%
Test accuracy: 34.3%


Validation - test accuracy ~ 2.2 %

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [65]:
def dropout_forward_pass(dataset, weights, biases, keep_prob):
    relu_hidden_layer_1 = tf.nn.relu(tf.add(tf.matmul(dataset, weights['h1']), biases['b1']))
    relu_hidden_layer_drop_1 = tf.nn.dropout(relu_hidden_layer_1, keep_prob)
    
    #relu_hidden_layer_2 = tf.nn.relu(tf.add(tf.matmul(relu_hidden_layer_1, weights['h2']), biases['b2']))
    # relu_hidden_layer_drop_2 = tf.nn.dropout(relu_hidden_layer_2, keep_prob)
    
    out_layer = tf.matmul(relu_hidden_layer_drop_1, weights['out']) + biases['out']
    return out_layer

mlp_dropout_graph = tf.Graph()
with mlp_dropout_graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    keep_prob = tf.placeholder("float")
    
    # Store layers weight & bias
    n_input = image_size * image_size
    n_hidden_1 = 1024
    n_hidden_2 = 32
    n_classes = num_labels
    
    weights = {
        'h1': tf.Variable(tf.truncated_normal([n_input, n_hidden_1])),
        #'h2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2])),
        'out': tf.Variable(tf.truncated_normal([n_hidden_1, n_classes]))
    }
    
    biases = {
        'b1': tf.Variable(tf.zeros([n_hidden_1])),
        #'b2': tf.Variable(tf.zeros([n_hidden_2])),
        'out': tf.Variable(tf.zeros([n_classes]))
    }
    
    out_layer = dropout_forward_pass(tf_train_dataset, weights, biases, 0.5)
    
    # Yes, those probably should be used as placeholder values, but ... FIXME
    test_out_layer = dropout_forward_pass(tf_test_dataset, weights, biases, 0.5)
    valid_out_layer = dropout_forward_pass(tf_valid_dataset, weights, biases, 0.5)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(out_layer, tf_train_labels)) # Softmax loss
    
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.9)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
train_and_test(mlp_dropout_graph, train_dataset, 3000)

initialization finished
Minibatch loss at step 0: 586.713745
Minibatch accuracy: 7.8%
Validation accuracy: 33.0%
Test accuracy: 35.5%
Minibatch loss at step 500: 24.694050
Minibatch accuracy: 79.7%
Validation accuracy: 75.7%
Test accuracy: 82.2%
Minibatch loss at step 1000: 13.791576
Minibatch accuracy: 75.0%
Validation accuracy: 73.8%
Test accuracy: 80.1%
Minibatch loss at step 1500: 14.264048
Minibatch accuracy: 67.2%
Validation accuracy: 71.4%
Test accuracy: 78.6%
Minibatch loss at step 2000: 6.245362
Minibatch accuracy: 74.2%
Validation accuracy: 74.6%
Test accuracy: 81.3%
Minibatch loss at step 2500: 12.325098
Minibatch accuracy: 72.7%
Validation accuracy: 72.9%
Test accuracy: 80.0%
